In [1]:
import nibabel as nib
import numpy as np
import glob

/Users/paddyslator/anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
datadir='/Users/paddyslator/OneDrive - University College London/data/bayes-dmipy-paddy/AMICO/NODDI'

In [3]:
#load the AMICO masks
fibdir = nib.load(datadir + '/FIT_dir.nii')
fibdir = fibdir.get_data()
OD = nib.load(datadir + '/FIT_OD.nii')
OD = OD.get_data()
ISOVF = nib.load(datadir + '/FIT_ISOVF.nii')
ISOVF = ISOVF.get_data()
ICVF = nib.load(datadir + '/FIT_ICVF.nii')
ICVF = ICVF.get_data()

In [5]:
def cart2mu(xyz):
    import numpy as np

    shape = xyz.shape[:-1]
    mu = np.zeros(np.r_[shape, 2])
    r = np.linalg.norm(xyz, axis=-1)
    mu[..., 0] = np.arccos(xyz[..., 2] / r)  # theta
    mu[..., 1] = np.arctan2(xyz[..., 1], xyz[..., 0])
    mu[r == 0] = 0, 0
    return mu

In [6]:
#convert cartesian to polar 
fibdirpolar = cart2mu(fibdir)

/Users/paddyslator/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [7]:
nvox = np.prod(np.shape(OD)[0:3])


In [8]:
#put the data in voxel form
muvox = np.reshape(fibdirpolar, (nvox, 2))
ODvox = np.reshape(OD, (nvox))
ISOVFvox = np.reshape(ISOVF, (nvox)) #ball?
ICVFvox = np.reshape(ICVF, (nvox)) #stick?




In [9]:
#put in the format that bayesian fit needs 
parameter_vector_lsq={}
parameter_vector_lsq['SD1WatsonDistributed_1_SD1Watson_1_mu'] = muvox
parameter_vector_lsq['SD1WatsonDistributed_1_SD1Watson_1_odi'] = ODvox
parameter_vector_lsq['SD1WatsonDistributed_1_partial_volume_0'] = ICVFvox
parameter_vector_lsq['partial_volume_0'] = ISOVFvox
parameter_vector_lsq['partial_volume_1'] = 1 - ISOVFvox

